# Text Rank from Scratch

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import contractions
nltk.download('punkt') # one time execution

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dwight\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
%autosave 120
pd.set_option('display.max_colwidth',1000)

Autosaving every 120 seconds


# Load Glove Embeddings

In [3]:
import zipfile

GLOVE_DIR = 'data/glove/'
GLOVE_ZIP = GLOVE_DIR + 'glove.6B.50d.zip'

zip_ref = zipfile.ZipFile(GLOVE_ZIP, 'r')
zip_ref.extractall(GLOVE_DIR)
zip_ref.close()

In [4]:
import numpy as np

def load_glove_vectors(fn):
    print("Loading Glove Model")
    with open( fn,'r', encoding='utf8') as glove_vector_file:
        model = {}
        for line in glove_vector_file:
            parts = line.split()
            word = parts[0]
            embedding = np.array([float(val) for val in parts[1:]])
            model[word] = embedding
        print("Loaded {} words".format(len(model)))
    return model

glove_vectors = load_glove_vectors('data/glove/glove.6B.50d.txt')

Loading Glove Model
Loaded 400000 words


# Read Articles

In [5]:
articles = pd.read_csv("data/tennis_articles_v4.csv")

In [6]:
articles.head(2)

,article_id,article_text,source
0,1,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...",https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours w...",http://www.tennis.com/pro-game/2018/10/copil-stuns-5th-ranked-zverev-to-reach-swiss-indoors-final/77721/


In [7]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dwight\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize

CLEAN_PATTERN = r'[^a-zA-z\s]'

def clean(word):
    return re.sub(CLEAN_PATTERN, '', word)

def clean_sentence(sentence):
    sentence = [clean(word) for word in sentence]
    return [word for word in sentence if word]

def clean_sentences(sentences):
    return [clean_sentence(sentence) for sentence in sentences]

def lower(sentence):
    return [word.lower() for word in sentence]

def remove_stopwords(sentence):
    words = [word for word in sentence if word not in stop_words]
    return [word for word in words if len(word) >0]

def tokenize_words(sentences):
    return [word_tokenize(sentence) 
              for sentence in sentences]

def fix_contractions(sentences):
    return [contractions.fix(sentence) for sentence in sentences]

articles['SentencesInArticle'] = articles.article_text.apply(sent_tokenize)
articles['WordsInSentences'] = articles.SentencesInArticle \
              .apply(fix_contractions)\
              .apply(lower)\
              .apply(tokenize_words)\
              .apply(remove_stopwords)\
              .apply(clean_sentences)

In [9]:
articles = articles[['SentencesInArticle', 'WordsInSentences']]
articles.head(2)

,SentencesInArticle,WordsInSentences
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,..."
1,"[BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday., Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday., Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1., He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand., He clinched on his fourth chance when Medvedev netted from the baseline., Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal., The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent., Copil has two after also beating No., 6 Marin Cilic in the second round., Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1...","[[basel, switzerland, ap, roger, federer, advanced, to, the, th, swiss, indoors, final, of, his, career, by, beating, seventhseeded, daniil, medvedev, on, saturday], [seeking, a, ninth, title, at, his, hometown, event, and, a, th, overall, federer, will, play, thranked, marius, copil, on, sunday], [federer, dominated, the, thranked, medvedev, and, had, his, first, matchpoint, chance, to, break, serve, again, at], [he, then, dropped, his, serve, to, love, and, let, another, match, point, slip, in, medvedev, s, next, service, game, by, netting, a, backhand], [he, clinched, on, his, fourth, chance, when, medvedev, netted, from, the, baseline], [copil, upset, expectations, of, a, federer, final, against, alexander, zverev, in, a, win, over, the, fifthranked, german, in, the, earlier, semifinal], [the, romanian, aims, for, a, first, title, after, arriving, at, basel, without, a, career, win, over, a, top, opponent], [copil, has, two, after, al

## Articles

In [10]:
articles.head(2)

,SentencesInArticle,WordsInSentences
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,..."
1,"[BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday., Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday., Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1., He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand., He clinched on his fourth chance when Medvedev netted from the baseline., Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal., The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent., Copil has two after also beating No., 6 Marin Cilic in the second round., Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1...","[[basel, switzerland, ap, roger, federer, advanced, to, the, th, swiss, indoors, final, of, his, career, by, beating, seventhseeded, daniil, medvedev, on, saturday], [seeking, a, ninth, title, at, his, hometown, event, and, a, th, overall, federer, will, play, thranked, marius, copil, on, sunday], [federer, dominated, the, thranked, medvedev, and, had, his, first, matchpoint, chance, to, break, serve, again, at], [he, then, dropped, his, serve, to, love, and, let, another, match, point, slip, in, medvedev, s, next, service, game, by, netting, a, backhand], [he, clinched, on, his, fourth, chance, when, medvedev, netted, from, the, baseline], [copil, upset, expectations, of, a, federer, final, against, alexander, zverev, in, a, win, over, the, fifthranked, german, in, the, earlier, semifinal], [the, romanian, aims, for, a, first, title, after, arriving, at, basel, without, a, career, win, over, a, top, opponent], [copil, has, two, after, al

In [11]:
VECTOR_SIZE = 50
EMPTY_VECTOR = np.zeros(VECTOR_SIZE)

def sentence_vector(sentence):
    return sum([glove_vectors.get(word, EMPTY_VECTOR) 
                  for word in sentence])/len(sentence)

def sentences_to_vectors(sentences):
    return [sentence_vector(sentence) 
              for sentence in sentences]

## Articles With Sentence Vectors

In [12]:
articles['SentenceVector'] = \
        articles.WordsInSentences.apply(sentences_to_vectors)

## Similarity Matrix

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity_matrix(sentence_vectors):
    sim_mat = np.zeros([len(sentence_vectors), len(sentence_vectors)])
    for i in range(len(sentence_vectors)):
        for j in range(len(sentence_vectors)):
            element_i = sentence_vectors[i].reshape(1,VECTOR_SIZE)
            element_j = sentence_vectors[j].reshape(1,VECTOR_SIZE)
            sim_mat[i][j] = cosine_similarity(element_i,
                                              element_j)[0,0]
    return sim_mat

In [26]:
articles['SimMatrix'] = \
     articles.SentenceVector.apply(similarity_matrix)

In [27]:
import networkx as nx

def compute_graph(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

## Graph

In [28]:
articles['Graph'] = articles.SimMatrix.apply(compute_graph)

In [29]:
articles.head(2)

,SentencesInArticle,WordsInSentences,SentenceVector,SimMatrix,Graph,Summary
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,...","[[0.07161230769230773, 0.5263353076923079, -0.22625407692307692, 0.4190584615384615, -0.0952153076923077, -0.10156600000000002, -0.40362538461538466, 0.400633476923077, -0.35831371000000006, -0.11500546153846154, 0.11607930769230769, -0.146259, -0.6847891923076924, 0.014327538461538456, 0.783284653846154, 0.046932615384615385, -0.10918245384615388, 0.054596461538461545, -0.29416438461538463, -0.1929796923076923, -0.2468181230769231, 0.3106658461538461, 0.011112153846153848, 0.30192715384615376, 0.12148384615384614, -1.328529230769231, -0.2920881538461539, -0.07721707692307692, -0.3258306923076923, -0.36910523076923074, 2.7283676923076925, 0.24652538461538465, -0.041018076923076914, 0.07203453846153846, 0.022849225384615382, -0.01111693076923078, 0.2808099230769231, 0.4716086153846154, -0.020941546153846143, -0.6126247692307694, -0.007031076923076909, -0.08967269230769231, 0.18087161538461535, 0.06510876923076923, 0.1128052307692308, -0.027662769230769227, -0.06883483846153847, -0.3...","[[0.0, 0.8804204422835109, 0.8400741992486218, 0.9129596149615926, 0.8445816500505545, 0.8497893583785374, 0.876852166283095, 0.886258925167321, 0.876688184230672, 0.9263429106619361, 0.8600162727238754, 0.88957833049645, 0.8313178033584998, 0.8565959297474462, 0.8824241286286072, 0.8868385761498727, 0.8136190246763892], [0.8804204422835109, 0.0, 0.9408964676641778, 0.9686965549113967, 0.8942781393973298, 0.9586334393985563, 0.9585076101382632, 0.9730773874995189, 0.987312380978821, 0.9125283526364883, 0.9594498058208407, 0.9640488970318378, 0.9632944418681554, 0.9732690670320119, 0.9616582246780772, 0.9741510958493371, 0.9564473923061918], [0.8400741992486218, 0.9408964676641778, 0.0, 0.9303313796847564, 0.9448434579223793, 0.9570165239566274, 0.9414391114508265, 0.9183752804522205, 0.9549370028865451, 0.840775322936442, 0.9105576429589421, 0.9547127695525695, 0.9563604609626246, 0.9106663731073268, 0

## Ranked Sentences

In [30]:
def get_ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s) 
                         for i,s in enumerate(sentences)), 
                                reverse=True)
    top_n_sentences = [sentence 
                        for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

articles['Summary'] = articles.apply(lambda d: 
                                     get_ranked_sentences(d.SentencesInArticle, 
                                                          d.Graph), axis=1)

## Summary

In [31]:
articles.loc[0].Summary

"I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players. But ultimately tennis is just a very small part of what we do. I think everyone just thinks because we're tennis players we should be the greatest of friends."

In [32]:
articles.loc[1].Summary

'Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent.'